# Bostrom Revenue and Price Model

In [1]:
from IPython.display import display, Markdown

from src.data_extractors import get_pools, get_prices
from config import BOSTROM_LCD_CLIENT

BASE_INVESTMINTING_AMOUNT_AMPERE = 100_000_000
BASE_INVESTMINTING_AMOUNT_VOLT = 1_000_000_000
BASE_INVESTMINTING_PERIOD_AMPERE = 30
BASE_INVESTMINTING_PERIOD_VOLT = 30

VALIDATOR_COMMISSION = 0.05
NUMBER_DEGEN_ITERATIONS = 3

## Optimal State
PRICE_HYDROGEN_OPTIMAL = 0.2
REVENUE_HYDROGEN_OPTIMAL = 0.4
BOOT_REVENUE_OPTIMAL = 0.2  # stake + restake

## Actual State

In [2]:
staked_boot =  BOSTROM_LCD_CLIENT.staking.pool().bonded_tokens.amount
supply_boot = [int(_coin['amount']) for _coin in BOSTROM_LCD_CLIENT.bank.total()[0].to_data() if _coin['denom'] == 'boot'][0]
inflation = float(BOSTROM_LCD_CLIENT.mint.inflation())
community_tax = float(BOSTROM_LCD_CLIENT.distribution.parameters()['community_tax'])

revenue_boot = supply_boot / staked_boot * inflation * (1 - VALIDATOR_COMMISSION) * (1 - community_tax)
revenue_with_restake_boot = (1 + revenue_boot/365) ** 365 - 1

price_df = get_prices(get_pools(networks=['bostrom', 'osmosis']), zero_fee=True)

price_ampere_actual = price_df.loc['hydrogen', 'milliampere'] * 1_000
price_volt_actual = price_df.loc['hydrogen', 'millivolt'] * 1_000
price_hydrogen_actual = price_df.loc['boot', 'hydrogen']

## Revenue Strategies
### Degen
Stake BOOT with restake -> Buy BOOT for HYDROGEN -> Stake BOOT with restake - 3 times

In [3]:
degen_revenue_actual = degen_revenue_optimal = 0
staked_boot_actual = staked_boot_optimal = 1

for _ in range(NUMBER_DEGEN_ITERATIONS):
    degen_revenue_actual +=  staked_boot_actual * revenue_with_restake_boot
    staked_boot_actual *= price_hydrogen_actual

    degen_revenue_optimal +=  staked_boot_optimal * BOOT_REVENUE_OPTIMAL
    staked_boot_optimal *= PRICE_HYDROGEN_OPTIMAL

### Staking + Investminting

Stake BOOT with retake -> Investmint HYDROGEN

In [4]:
hydrogen_revenue_actual = \
    (1 * 365 / (BASE_INVESTMINTING_AMOUNT_VOLT * BASE_INVESTMINTING_PERIOD_VOLT) * price_volt_actual +
     1 * 365 / (BASE_INVESTMINTING_AMOUNT_AMPERE * BASE_INVESTMINTING_PERIOD_AMPERE) * price_ampere_actual) / 2
staking_investminting_revenue_actual = revenue_with_restake_boot + price_hydrogen_actual * hydrogen_revenue_actual
staking_investminting_revenue_optimal = BOOT_REVENUE_OPTIMAL + PRICE_HYDROGEN_OPTIMAL * REVENUE_HYDROGEN_OPTIMAL

## Price Model

In [5]:
price_ampere_optimal = BASE_INVESTMINTING_AMOUNT_AMPERE * BASE_INVESTMINTING_PERIOD_AMPERE / 365 * REVENUE_HYDROGEN_OPTIMAL
price_volt_optimal = BASE_INVESTMINTING_AMOUNT_VOLT * BASE_INVESTMINTING_PERIOD_VOLT / 365 * REVENUE_HYDROGEN_OPTIMAL

## Display data

In [6]:
display(Markdown('''
# Bostrom Revenue and Price Model
## Actual state'''))
print(f'BOOT Supply {supply_boot:>22,}\n'
      f'BOOT Staked {staked_boot:>22,}\n'
      f'BOOT Inflation  {inflation:28.2%}\n'
      f'BOOT Inflation Community Tax  {community_tax:14.2%}\n'
      f'BOOT Staking Revenue without restake  {revenue_boot:3.2%}\n'
      f'BOOT Staking Revenue with restake     {revenue_with_restake_boot:3.2%}\n')

display(Markdown('''
## Revenue Strategies
### Degen
Stake BOOT with restake -> Buy BOOT for HYDROGEN -> Stake BOOT with restake - 3 times'''))
print(f'Degen revenue actual {degen_revenue_actual:7.1%}\n'
      f'Degen revenue optimal {degen_revenue_optimal:6.1%}')

display(Markdown('''
### Staking + Investminting

Stake BOOT with retake -> Investmint HYDROGEN'''))
print(f'Staking + Investminting revenue actual {staking_investminting_revenue_actual:7.1%}\n'
      f'Staking + Investminting revenue optimal {staking_investminting_revenue_optimal:6.1%}')

display(Markdown('''
### Investminting
Buy HYDROGEN for BOOT -> Investmint HYDROGEN'''))
print(f'Investminting revenue actual {hydrogen_revenue_actual:7.1%}\n'
      f'Investminting revenue optimal {REVENUE_HYDROGEN_OPTIMAL:6.1%}')

display(Markdown('''
## Price Model'''))
print(f'AMPERE price actual {price_ampere_actual:>,.0f} H optimal {price_ampere_optimal:>,.0f} H can decrease by {price_ampere_actual/price_ampere_optimal:.2f} times')
print(f'VOLT price actual {price_volt_actual:>,.0f} H optimal {price_volt_optimal:>,.0f} H can decrease by {price_volt_actual/price_volt_optimal:.2f} times')
print(f'H price actual {price_hydrogen_actual:.3f} BOOT optimal {PRICE_HYDROGEN_OPTIMAL} BOOT can decrease by {price_hydrogen_actual/PRICE_HYDROGEN_OPTIMAL:.2f} times')

print(f"OSMO price"
      f"\n\t- in bostrom {price_df.loc['hydrogen', 'uosmo in bostrom'] * 1_000_000:>,.0f} H"
      f"\n\t- in osmosis {price_df.loc['boot in osmosis', 'uosmo'] * price_df.loc['hydrogen', 'boot'] * 1_000_000:>,.0f} H")
print(f"ATOM price"
      f"\n\t- in bostrom {price_df.loc['hydrogen', 'uatom in bostrom'] * 1_000_000:>,.0f} H"
      f"\n\t- in osmosis {price_df.loc['boot in osmosis', 'uatom in osmosis'] * price_df.loc['hydrogen', 'boot'] * 1_000_000:>,.0f} H")



# Bostrom Revenue and Price Model
## Actual state

BOOT Supply  1,147,683,326,457,073
BOOT Staked    272,231,853,177,508
BOOT Inflation                         5.46%
BOOT Inflation Community Tax          10.00%
BOOT Staking Revenue without restake  19.68%
BOOT Staking Revenue with restake     21.74%




## Revenue Strategies
### Degen
Stake BOOT with restake -> Buy BOOT for HYDROGEN -> Stake BOOT with restake - 3 times

Degen revenue actual   29.2%
Degen revenue optimal  24.8%



### Staking + Investminting

Stake BOOT with retake -> Investmint HYDROGEN

Staking + Investminting revenue actual   32.8%
Staking + Investminting revenue optimal  28.0%



### Investminting
Buy HYDROGEN for BOOT -> Investmint HYDROGEN

Investminting revenue actual   40.7%
Investminting revenue optimal  40.0%



## Price Model

AMPERE price actual 3,408,557 H optimal 3,287,671 H can decrease by 1.04 times
VOLT price actual 32,888,371 H optimal 32,876,712 H can decrease by 1.00 times
H price actual 0.271 BOOT optimal 0.2 BOOT can decrease by 1.36 times
OSMO price
	- in bostrom 116,755,716 H
	- in osmosis 115,145,339 H
ATOM price
	- in bostrom 2,183,366,116 H
	- in osmosis 2,179,960,314 H
